# Friends 감정분석 with LightGBM
참고(링크) : 

# 준비
패키지, 파라미터 세팅

In [1]:
import pandas as pd
import numpy as np
import os
import re
import random
import time
import datetime
import json

In [2]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'

TEST_SIZE = 0.2
RANDOM_SEED = 42

# 데이터 로드

In [3]:
def jsonToDf(file_name):
    with open(file_name, encoding = 'utf-8', mode = 'r') as file:
        json_array = json.load(file)
  
    result = pd.DataFrame.from_dict(json_array[0])

    is_first = True
    for array in json_array:
        if is_first:
            is_first = False
            continue
    
        temp_df = pd.DataFrame.from_dict(array)
        result = result.append(temp_df, ignore_index = True)

    return result

In [4]:
train_df = jsonToDf(DATA_IN_PATH+'friends_train.json')  # 학습용
dev_df = jsonToDf(DATA_IN_PATH+'friends_dev.json')  # 검증용
test_df = pd.read_csv(DATA_IN_PATH+'en_data.csv')  # 테스트(캐글) 데이터

In [5]:
print(train_df.shape)
print(dev_df.shape)
print(test_df.shape)

(10561, 4)
(1178, 4)
(1623, 5)


In [6]:
train_df.head()

,speaker,utterance,emotion,annotation
0,Chandler,also I was the point person on my companys tr...,neutral,4100000
1,The Interviewer,You mustve had your hands full.,neutral,5000000
2,Chandler,That I did. That I did.,neutral,5000000
3,The Interviewer,So lets talk a little bit about your duties.,neutral,5000000
4,Chandler,My duties? All right.,surprise,2000030


In [7]:
dev_df.head()

,speaker,utterance,emotion,annotation
0,Phoebe,"Oh my God, hes lost it. Hes totally lost it.",non-neutral,0002120
1,Monica,What?,surprise,1000130
2,Ross,"Or! Or, we could go to the bank, close our acc...",neutral,3000200
3,Chandler,Youre a genius!,joy,0500000
4,Joey,"Aww, man, now we wont be bank buddies!",sadness,0040100


In [8]:
test_df.head()

,id,i_dialog,i_utterance,speaker,utterance
0,0,0,0,Phoebe,"Alright, whadyou do with him?"
1,1,0,1,Monica,Oh! You're awake!
2,2,0,2,Joey,Then you gotta come clean with Ma! This is not...
3,3,0,3,Mr. Tribbiani,"Yeah, but this is"
4,4,0,4,Joey,I don't wanna hear it! Now go to my room!


# 데이터 전처리

In [9]:
def data_cleansing(train_data, dev_data, test_data, con=0):  # 0: 전처리 없음(null값 공백 치환), 1: 숫자 제거, 2: 특수문자 제거
    global train_df
    global dev_df
    global test_df
    
    train_data = jsonToDf(DATA_IN_PATH+'friends_train.json')  # 학습용
    dev_data = jsonToDf(DATA_IN_PATH+'friends_dev.json')  # 검증용
    test_data = pd.read_csv(DATA_IN_PATH+'en_data.csv')  # 테스트(캐글) 데이터
   
    train_data.drop(['speaker','annotation'], axis=1, inplace=True)
    dev_data.drop(['speaker','annotation'], axis=1, inplace=True)
    test_data.drop(['i_dialog','i_utterance','speaker'], axis=1, inplace=True)

    train_df = train_data.copy()
    dev_df = dev_data.copy()
    test_df = test_data.copy()
    
    if con == 1:  # 숫자 제거 & Null 치환(공백), row 제거 없음
        # 정규 표현식을 이용해 숫자를 공백으로 변경(정규 표현식으로 \d는 숫자를 의미)
        train_df = train_df.fillna(' ')
        train_df['utterance'] = train_df['utterance'].apply( lambda x : re.sub(r"\d+", " ", x) )  # 숫자 제거
        dev_df = dev_df.fillna(' ')
        dev_df['utterance'] = dev_df['utterance'].apply( lambda x : re.sub(r"\d+", " ", x) )  # 숫자 제거
        test_df = test_df.fillna(' ')
        test_df['utterance'] = test_df['utterance'].apply( lambda x : re.sub(r"\d+", " ", x) )  # 숫자 제거
        
    elif con == 2:  # 특수문자 제거 & Null 치환(공백), row 제거 없음
        train_df = train_df.fillna(' ')
        train_df['utterance'] = train_df['utterance'].str.replace("[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\x92]","") # 정규 표현식 수행(특수문자 제거)
        dev_df = dev_df.fillna(' ')
        dev_df['utterance'] = dev_df['utterance'].str.replace("[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\x92]","") # 정규 표현식 수행(특수문자 제거)
        test_df = test_df.fillna(' ')
        test_df['utterance'] = test_df['utterance'].str.replace("[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\x92]","") # 정규 표현식 수행(특수문자 제거)
        
    else:
        train_df = train_df.fillna(' ')
        dev_df = dev_df.fillna(' ')
        test_df = test_df.fillna(' ')
    
    print('전처리 후 학습 데이터 :',train_df.shape)
    print('전처리 후 검증 데이터 :',dev_df.shape)
    print('전처리 후 테스트 데이터 :',test_df.shape)


In [10]:
# 데이터 전처리 선택
data_cleansing(train_df, dev_df, test_df, con=0)  # 0: 전처리 없음(null값 공백 치환), 1: 숫자 제거, 2: 특수문자 제거

전처리 후 학습 데이터 : (10561, 2)
전처리 후 검증 데이터 : (1178, 2)
전처리 후 테스트 데이터 : (1623, 2)


In [11]:
train_df.head()

,utterance,emotion
0,also I was the point person on my companys tr...,neutral
1,You mustve had your hands full.,neutral
2,That I did. That I did.,neutral
3,So lets talk a little bit about your duties.,neutral
4,My duties? All right.,surprise


In [12]:
dev_df.head()

,utterance,emotion
0,"Oh my God, hes lost it. Hes totally lost it.",non-neutral
1,What?,surprise
2,"Or! Or, we could go to the bank, close our acc...",neutral
3,Youre a genius!,joy
4,"Aww, man, now we wont be bank buddies!",sadness


In [13]:
test_df.head()

,id,utterance
0,0,"Alright, whadyou do with him?"
1,1,Oh! You're awake!
2,2,Then you gotta come clean with Ma! This is not...
3,3,"Yeah, but this is"
4,4,I don't wanna hear it! Now go to my room!


In [14]:
# Friends 감정 딕셔너리 생성
emoset = {'non-neutral': 0,
          'neutral': 1, 
          'joy': 2,
          'sadness': 3,
          'fear': 4,
          'anger': 5,
          'surprise': 6,
          'disgust': 7}

# # 사이킷런을 활용한 레이블인코딩
# from sklearn.preprocessing import LabelEncoder

# items = ['non-neutral','neutral','joy','sadness','fear','anger','surprise','disgust']

# # LabelEncoder를 객체로 생성한 후, fit()과 transform()으로 레이블 인코딩 수행
# encoder = LabelEncoder()
# encoder.fit(items)
# labels = encoder.transform(items)

# print('인코딩 클래스 :', encoder.classes_)
# print('인코딩 변환값 :', labels)

In [15]:
# 'label' 컬럼 신규
#for emo in train_df['emotion'][:10]:  print(emo, emoset[emo])
train_df['label'] = [emoset[emo] for emo in train_df['emotion']]
dev_df['label'] = [emoset[emo] for emo in dev_df['emotion']]

In [16]:
train_df.head()

,utterance,emotion,label
0,also I was the point person on my companys tr...,neutral,1
1,You mustve had your hands full.,neutral,1
2,That I did. That I did.,neutral,1
3,So lets talk a little bit about your duties.,neutral,1
4,My duties? All right.,surprise,6


In [17]:
dev_df.head()

,utterance,emotion,label
0,"Oh my God, hes lost it. Hes totally lost it.",non-neutral,0
1,What?,surprise,6
2,"Or! Or, we could go to the bank, close our acc...",neutral,1
3,Youre a genius!,joy,2
4,"Aww, man, now we wont be bank buddies!",sadness,3


In [18]:
# 학습, 검증(Origianl Testset) 데이터 레이블 넘파이 배열 처리
y_train = np.array(train_df['label'])
y_dev = np.array(dev_df['label'])

# TF-IDF를 활용한 벡터화

Bag of Words 모델은 문서가 가지는 모든 단어를 문맥이나 순서를 무시하고 일괄적으로 단어에 대해 빈도 값을 부여해 피처 값을 추출하는 모델이다. 문서 내 모든 단어를 한꺼번에 봉투(bag) 안에 넣은 뒤에 흔들어서 섞는다는 의미로 Bag of Words(BOW) 모델이라고 한다.<br>

머신러닝 알고리즘은 일반적으로 숫자형 피처를 데이터로 입력받아 동작하기 때문에 텍스트 데이터는 머신러닝 알고리즘에 바로 입력할 수 없다. 따라서 텍스트는 특정 의미를 가지는 숫자형 값인 벡터 값으로 변환해야 하고, 이러한 변환을 피처 벡터화라고 한다. 일반적으로 BOW의 피처 벡터화는 (1) 카운트 벡터화, (2) TF-IDF 벡터화 두 가지 방식이 있다.

사이킷런의 CountVertorizer, TfidfVectorizer를 이용해 텍스트를 피처 단위로 벡터화해 변환하고 CSR 형태의 희소 행렬을 반환한다. 단순하게 피처 벡터화만 수행하는게 아니라 소문자 일괄 변환, 토큰화, 스톱 워드 필터링 등의 텍스트 전처리도 함께 수행할 수 있다.

벡터화의 결과는 대부분의 값이 0을 가지는 희소 행렬이며 메모리 공간을 효율적으로 사용하기 위한 CSR(Compressed Sparse Row) 형태이다. CountVertorizer, TfidfVectorize를 적용할 때는 반드시 학습 데이터를 이용해 fit()이 수행된 객체를 통해 테스트 데이터를 변환(transform)해야 한다. 그래야만 학습 시 사용된 피처 개수와 예측 시 사용할 피처 개수를 동일하게 맞출 수 있다.

일반적으로 문서 내에 텍스트가 많고 많은 문서를 가지는 텍스트 분석에서는 카운트 벡터화보다 TF-IDF 벡터화가 좋은 예측 결과를 도출한다.

In [19]:
import time
from sklearn.feature_extraction.text import TfidfVectorizer

start = time.time()  # 시작 시간 저장

# TfidfVectorizer를 이용해 학습 데이터를 TF-IDF 값으로 피처 변환
# TfidfVectorizer 클래스의 스톱 워드를 'english'로, ngram_range는 (1, 2), max_df=700으로 설정한다.
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=700)
vectorizer.fit(train_df['utterance'])
train_data_features = vectorizer.transform(train_df['utterance'])

# 학습 데이터를 적용한 TfidfVectorizer를 이용해 검증 데이터를 TF-IDF 값으로 피처 변환
dev_data_features = vectorizer.transform(dev_df['utterance'])

# 학습 데이터를 적용한 TfidfVectorizer를 이용해 테스트 데이터를 TF-IDF 값으로 피처 변환
test_data_features = vectorizer.transform(test_df['utterance'])

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
print()
print(train_data_features.shape)
print(dev_data_features.shape)
print(test_data_features.shape)
print(type(train_data_features))

time : 0.2278900146484375

(10561, 25448)
(1178, 25448)
(1623, 25448)
<class 'scipy.sparse.csr.csr_matrix'>


# (Baseline) 학습에서 검증 데이터 분리 후 모델 학습

In [20]:
from sklearn.model_selection import train_test_split

train_input, eval_input, train_label, eval_label = train_test_split(train_data_features, y_train, test_size=TEST_SIZE, random_state=RANDOM_SEED)

In [21]:
from lightgbm import LGBMClassifier

start = time.time()  # 시작 시간 저장

# 랜덤 포레스트 사용해서 감성 분석 분류 수행
clf = LGBMClassifier(random_state=RANDOM_SEED)
clf.fit(train_input, train_label)

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 

time : 4.294263124465942


In [22]:
print("Accuracy: %f" % clf.score(eval_input, eval_label))  # 검증 함수로 정확도 측정

Accuracy: 0.475154


# (Baseline) Original 테스트셋으로 성능 평가

In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.metrics import classification_report

preds = clf.predict(dev_data_features)
confusion = confusion_matrix(dev_df['label'], preds)

print('오차 행렬')
print(confusion,'\n')
print('LightGBM 정확도 :', accuracy_score(dev_df['label'], preds),'\n')

오차 행렬
[[ 26 156   7   4   0   0  17   4]
 [ 41 410  13   2   2   8  12   3]
 [  9  91  16   1   0   0   6   0]
 [ 13  35   1   9   0   2   1   1]
 [  2  20   0   2   1   0   3   1]
 [ 10  62   3   0   1   5   4   0]
 [ 16  92   3   4   1   3  32   0]
 [  6  13   1   0   0   1   1   1]] 

LightGBM 정확도 : 0.4244482173174873 



In [24]:
target_names = ['non-neutral','neutral','joy','sadness','fear','anger','surprise','disgust']

print(classification_report(dev_df['label'], preds, target_names=target_names))

              precision    recall  f1-score   support

 non-neutral       0.21      0.12      0.15       214
     neutral       0.47      0.84      0.60       491
         joy       0.36      0.13      0.19       123
     sadness       0.41      0.15      0.21        62
        fear       0.20      0.03      0.06        29
       anger       0.26      0.06      0.10        85
    surprise       0.42      0.21      0.28       151
     disgust       0.10      0.04      0.06        23

    accuracy                           0.42      1178
   macro avg       0.30      0.20      0.21      1178
weighted avg       0.37      0.42      0.35      1178



In [25]:
from sklearn.metrics import f1_score
f1 = f1_score(dev_df['label'], preds, average='weighted')
print('F1 Score : {:.5f}'.format(f1))

F1 Score : 0.35450


# (GridSearch) 모델 학습 with Pipeline

In [26]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

start = time.time()  # 시작 시간 저장

pipeline = Pipeline([
    ('tfidf_vect',TfidfVectorizer(stop_words='english')),
    ('lgb_clf', LGBMClassifier())
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(_) 2개를 연달아 붙여 
# GridSearchCV에 사용될 파라미터/하이퍼파라미터 이름과 값을 설정
params = { 'tfidf_vect__ngram_range': [(1,2),(1,3)]
           ,'tfidf_vect__max_df' : [300,700]
         }

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_pipe.fit(train_df['utterance'], y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(dev_df['utterance'])
print('Pipeline을 통한 XGBoost 예측 정확도는 {0:.3f}'.format(
      accuracy_score(y_dev, pred)))

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:   48.5s finished


{'tfidf_vect__max_df': 700, 'tfidf_vect__ngram_range': (1, 3)} 0.4602789378447927
Pipeline을 통한 XGBoost 예측 정확도는 0.438
time : 53.10881042480469


In [27]:
# 도출된 best 파라미터를 dictionary형 변수로 담는다
grid_dict = grid_cv_pipe.best_params_
grid_dict

{'tfidf_vect__max_df': 700, 'tfidf_vect__ngram_range': (1, 3)}

In [28]:
# TfidfVectorizer를 이용해 학습 데이터를 TF-IDF 값으로 피처 변환
# TfidfVectorizer 클래스의 스톱 워드를 'english'로, ngram_range, max_df는 GridSearchCV 통해 얻은 결과 입력
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=grid_dict.get('tfidf_vect__ngram_range'), \
                             max_df=grid_dict.get('tfidf_vect__max_df'))
vectorizer.fit(train_df['utterance'])
train_data_features = vectorizer.transform(train_df['utterance'])

# 학습 데이터를 적용한 TfidfVectorizer를 이용해 검증/테스트 데이터를 TF-IDF 값으로 피처 변환
dev_data_features = vectorizer.transform(dev_df['utterance'])
test_data_features = vectorizer.transform(test_df['utterance'])

In [29]:
# GridSearch 적용된 데이터셋을 통해 감성 분석 분류 수행

lgb_clf = LGBMClassifier(random_state= RANDOM_SEED)
lgb_clf.fit(train_data_features, y_train)

LGBMClassifier(random_state=42)

# (GridSearch+Pipeline) Original 테스트셋으로 성능 평가

In [30]:
preds = lgb_clf.predict(dev_data_features)

In [31]:
confusion = confusion_matrix(dev_df['label'], preds)

print('오차 행렬')
print(confusion,'\n')

print('Logistic Regression 정확도  :', accuracy_score(dev_df['label'], preds),'\n')

오차 행렬
[[ 29 153   9   1   2   2  18   0]
 [ 29 425  14   3   4   3  12   1]
 [  7  87  20   1   0   0   8   0]
 [ 14  36   3   7   0   1   1   0]
 [  4  19   0   1   1   1   2   1]
 [  8  65   5   0   1   3   2   1]
 [ 18  94   5   2   1   0  31   0]
 [  4  16   1   0   0   1   1   0]] 

Logistic Regression 정확도  : 0.4380305602716469 



In [32]:
target_names = ['non-neutral','neutral','joy','sadness','fear','anger','surprise','disgust']

print(classification_report(dev_df['label'], preds, target_names=target_names))

              precision    recall  f1-score   support

 non-neutral       0.26      0.14      0.18       214
     neutral       0.47      0.87      0.61       491
         joy       0.35      0.16      0.22       123
     sadness       0.47      0.11      0.18        62
        fear       0.11      0.03      0.05        29
       anger       0.27      0.04      0.06        85
    surprise       0.41      0.21      0.27       151
     disgust       0.00      0.00      0.00        23

    accuracy                           0.44      1178
   macro avg       0.29      0.19      0.20      1178
weighted avg       0.38      0.44      0.36      1178



In [33]:
from sklearn.metrics import f1_score
f1 = f1_score(dev_df['label'], preds, average='weighted')
print('F1 Score : {:.5f}'.format(f1))

F1 Score : 0.36158


# (참고) 제출 파일 생성

In [34]:
test_preds = lgb_clf.predict(test_data_features)
test_preds

array([1, 1, 1, ..., 1, 1, 1])

In [35]:
# 감정 레이블링 위해 기존 정수형 레이블로부터 역변환
emoset_reverse = dict(zip(emoset.values(),emoset.keys()))

# 테스트 데이터의 id, 리뷰 부분을 리스트 처리
ids = list(test_df['id'])
test_utterances = list(test_df['utterance'])

# 숫자로 인코딩 된 레이블을 감정명으로 재매칭
emo_preds = [emoset_reverse[label] for label in test_preds]

# 판다스 데이터프레임 통해 데이터 구성하여 output에 투입
output = pd.DataFrame( data={"Id": ids, "Predicted": emo_preds} )
output.head()

,Id,Predicted
0,0,neutral
1,1,neutral
2,2,neutral
3,3,neutral
4,4,neutral


In [36]:
# 해당 경로가 없으면 생성
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

# csv파일 생성
output.to_csv(DATA_OUT_PATH + "FRIENDS_LightGBM.csv", index = False)

### 캐글 제출 결과
**[2020.12.17]**<br>
0.45006 (공백 치환 외 전처리 수행 안한 경우)

# (참고) 리뷰 예측하기

In [37]:
# 입력을 리스트 형태로 넣어야 함
# 참고 : https://m.blog.naver.com/PostView.nhn?blogId=samsjang&logNo=220985170721&proxyReferer=https:%2F%2Fwww.google.com%2F
def sentiment_predict(new_sentence):
    emo_pres = dict(zip(emoset.values(),emoset.keys()))
    new_sentence = vectorizer.transform(new_sentence)  # TF-IDF 값으로 피처 변환
    print('예측: %s\n확률: %.3f%%' %(emo_pres[lgb_clf.predict(new_sentence)[0]], \
                                 np.max(lgb_clf.predict_proba(new_sentence))*100))

In [38]:
sentiment_predict(['how do you know?'])

예측: neutral
확률: 46.670%


In [39]:
sentiment_predict(['I love you!'])

예측: joy
확률: 44.584%


In [40]:
sentiment_predict(['I hate that!'])

예측: neutral
확률: 46.670%
